In [5]:
import base64
import json
res = "bnVsbA=="

output =base64.b64decode(res).decode('utf-8')
if output == "null":
    output = None

print(output)
print(type(output))

None
<class 'NoneType'>


In [ ]:
s3_url = "parent/child/testing/file.pdf"  

file_path = s3_url.split(".")[0:]


print(file_path)

file.pdf
file.html


In [7]:
import pandas as pd
from pandas import DataFrame
from lxml import html

def extract_tables_from_html(html_content: str) -> list[DataFrame] | None:
    tree = html.fromstring(html_content)
    tables = tree.xpath('//table')

    if not tables:
        return None

    tables_html = [html.tostring(table, method="html").decode() for table in tables]

    print(tables_html)
    dfs:list[DataFrame] = []
    for table_html in tables_html:
        try:
            df = pd.read_html(table_html)[0]
            dfs.append(df)
        except ValueError:
            continue

    return dfs if dfs else None
 
def save_tables_to_markdown(dfs: list[DataFrame], file_name: str) -> None:
    with open(file_name, 'w') as md_file:
        for i, df in enumerate(dfs):
            md_file.write(f"## Table {i + 1}\n\n")
            md_file.write(df.to_markdown())
            md_file.write("\n\n") 
 
with open('sample.html', 'r') as f:
    html_content = f.read()

dfs = extract_tables_from_html(html_content)

if dfs is not None:
    save_tables_to_markdown(dfs, 'tables.md')



In [26]:
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
import io

def extract_tables_from_html(html_content: str) -> list[DataFrame] | None:
    soup = BeautifulSoup(html_content, "html.parser")
    tables = soup.find_all("table")
    if not tables:
        print("There is no table")
        return None

    tables_html = [str(table) for table in tables]

    dfs:list[DataFrame] = []
    for table_html in tables_html:
        try:
            str_buffer = io.StringIO(table_html)
            df = pd.read_html(str_buffer)[0]
            dfs.append(df)
        except ValueError:
            print("There is no table")
            continue
    if dfs:
        return dfs
    return None

def save_tables_to_markdown(dfs: list[DataFrame], file_name: str) -> None:
    with open(file_name, "w") as md_file:
        for i, df in enumerate(dfs):
            md_file.write(f"## Table {i + 1}\n\n")
            md_file.write(df.to_markdown())
            md_file.write("\n\n")

with open("sample.html") as f:
    html_content = f.read()

dfs = extract_tables_from_html(html_content)
if dfs is not None:
    save_tables_to_markdown(dfs, "tables.md")


yes


NameError: name 'content' is not defined